# DataLoader in PyTorch

__DataLoader__ -- класс в PyTorch, который позволяет итеративно проходить по датасету, отвечает за оркестрацию всего процесса работы с датасетом.

In [ ]:
DataLoader(
  dataset, 
  batch_size=1, 
  shuffle=False, 
  sampler=None, 
  batch_sampler=None, 
  num_workers=0, 
  collate_fn=None, 
  pin_memory=False, 
  drop_last=False, 
  timeout=0, 
  worker_init_fn=None, 
  prefetch_factor=2, 
  persistent_workers=False
  )

- __dataset__ -- позволяет создать кастомные классы для работы с датасетом, где можно указать логику формирвоания батча.
- __sampler__ -- определяет порядок элементов из датасета, которые будут идти в батч, то есть это список индексов, объединенных в батч. Удобно переопределять, когда обучение распредленное.  
- __collate_fn__ -- позволяет сделать финальную предобработку над батчем данных. Если, например, в батч попали последовательности разных размеров, то после уже сбора батча, можно будет дополнить последовательности нулями относительно максимально длиной последовательности.



## Custom Dataset

In [1]:
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm_notebook

from torch.utils.data import DataLoader, Dataset, Sampler
from torch.utils.data.dataloader import default_collate

In [2]:
BATCH_SIZE = 128
EPOCHS = 100

In [3]:
class CustomDataset(Dataset):
    # Конструктор, где считаем датасет
    def __init__(self, dataset_path):
        with open(dataset_path, 'rb') as f:
            self.X, self.target = pickle.load(f)

        return
    
    # Переопределяем метод вычисление размера датасета
    def __len__(self):
        return len(self.X)

    # Переопределяем метод,
    # который достает по индексу наблюдение из датасет
    def __getitem__(self, idx):
        return self.X[idx], self.target[idx]

## Custom Sampler

In [4]:
class CustomSampler(Sampler):

    # Конструктор, где инициализируем индексы элементов
    def __init__(self, data):
        self.data_indices = np.arange(len(data))

        shuffled_indices = np.random.permutation(len(self.data_indices))

        self.data_indices = np.ascontiguousarray(self.data_indices)[shuffled_indices]

        return

    def __len__(self):
        return len(self.data_indices)

    # Возращает итератор,
    # который будет возвращать индексы из перемешанного датасета
    def __iter__(self):
        return iter(self.data_indices)

## Custom collate_fn

In [5]:
def collate(batch):
    return default_collate(batch)

In [6]:
def create_data_loader(train_dataset, train_sampler,
                       test_dataset, test_sampler):
    train_loader = DataLoader(dataset=train_dataset, sampler=train_sampler,
                              batch_size=BATCH_SIZE, collate_fn=collate,
                              shuffle=False)

    test_loader = DataLoader(dataset=test_dataset, sampler=test_sampler,
                             batch_size=BATCH_SIZE, collate_fn=collate,
                             shuffle=False)

    return train_loader, test_loader

In [7]:
!git clone https://github.com/RiskModellingResearch/DeepLearning_Winter22.git

Cloning into 'DeepLearning_Winter22'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 54 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (54/54), done.


In [7]:
# Создаем объекты Custom Dataset и Sampler
train_ds = CustomDataset('DeepLearning_Winter22/week_03/data/X_train_cat.pickle')
train_sampler = CustomSampler(train_ds.X)

test_ds = CustomDataset('DeepLearning_Winter22/week_03/data/X_test_cat.pickle')
test_sampler = CustomSampler(test_ds.X)

In [8]:
train_loader, test_loader = create_data_loader(train_ds, train_sampler, 
                                               test_ds, test_sampler)

In [9]:
def run_train():
    for epoch in tqdm_notebook(range(EPOCHS)):
        for features, labels in train_loader:
            pass
    return

In [10]:
run_train()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/100 [00:00<?, ?it/s]